In [1]:
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification
from pyspark.sql import functions as fn, Row
from pyspark import sql
import re
import time

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
flights_svm_clean = spark.read.parquet('flights_transformed.parquet')

In [3]:
# Make a training, validation, and test split #
training_df, validation_df, testing_df = flights_svm_clean.randomSplit([0.6, 0.3, 0.1], seed=0)

In [4]:
svm_model = classification.LinearSVC(featuresCol='features',labelCol='DEPARTURE_DELAY_index').fit(training_df)

In [6]:
# Transform it against the validation data #
svm_df = svm_model.transform(validation_df).select('prediction','DEPARTURE_DELAY_index').toPandas()

In [7]:
# Show the results #
evaluation_svm_df = pd.concat([svm_df['prediction'],svm_df['DEPARTURE_DELAY_index']], 
                          axis = 1, keys=['prediction','label'])
crosstab_svm_df = pd.crosstab(evaluation_svm_df.prediction, evaluation_svm_df.label)
display(crosstab_svm_df)
print("Accuracy: {:,.3f}%" .format((crosstab_svm_df[0][0] + crosstab_svm_df[1][1]) / crosstab_svm_df.values.sum() * 100))

label,0.0,1.0
prediction,,
0.0,12523,6149
1.0,3973,3791


Accuracy: 61.711%
